# <center>AMAL - TP 9</center>  
## <center>注意力机制</center>

### 引言（简要，参考课程）

在本次实验中，我们将研究**自注意力机制（self-attention）**。与前一次实验不同的是，本实验中的注意力机制目标并非生成一个固定大小的序列表示 $x_{1}^{(0)}, \ldots, x_{n}^{(0)}$，而是生成一个上下文相关的表示序列 $x_{1}^{(L)}, \ldots, x_{n}^{(L)}$。其中，$x_{i}^{(L)}$ 是经过上下文建模后的第 $i$ 个元素：序列中的每个元素通过使用自注意力机制（问题由该元素的表示决定）独立计算自身的表示。这一过程将重复 $L$ 次，以获得更优的最终表示。

具体来说，经过第 $l$ 层后，序列中每个元素 $x_{i}^{(l)}$ 会通过三组线性层生成三个表示：一个 **查询（query）** 表示 $q_{\theta(l)}(x_{i}^{(l)})$，一个 **键（key）** 表示 $k_{\theta(l)}(x_{i}^{(l)})$，以及一个 **值（value）** 表示 $v_{\theta(l)}(x_{i}^{(l)})$。通过查询表示和键表示计算元素 $i$ 对元素 $j$ 的注意力权重 $\alpha_{ij}^{(l)}$：

$$
\log p(a_{ij}^{(l)}) = \text{constante} + \frac{1}{\sqrt{d}} q_{\theta^{(l)}}(x_{i}^{(l)}) \cdot k_{\theta^{(l)}}(x_{j}^{(l)})
$$

其中，$d$ 是表示的维度，因子 $d^{-1/2}$ 用于确保梯度的幅值在不同维度下保持一致。

在下一层中，元素 $x_{i}^{(l+1)}$ 的表示通过加权平均计算，权重由注意力权重决定，对应上一层的值表示：

$$
f_{\theta^{(l)}}(x_{i}^{(l)}; x_{1}^{(l)}, \ldots, x_{n}^{(l)}) = \sum_{j=1}^{n} p(a_{ij}^{(l)}) v_{\theta^{(l)}}(x_{j}^{(l)})
$$

函数 $f_{\theta^{(l)}}$ 是一个注意力函数（自注意力），其中查询、键和值分别是输入 $x_{i}^{(l)}$ 的线性函数。对于某一层 $l$，查询 $q$、键 $k$ 和值 $v$ 的参数对所有输入是相同的，但这些函数在不同层中是不同的（即，每一层计算查询、键和值的模块拥有独立的参数）。

通常，我们会在最后引入一层线性变换，并紧随其后应用一个非线性函数（例如 ReLU 函数），以获得 $x_{i}^{(l+1)}$ 的表示：

$$
x_i^{(l+1)} = g_{\theta^{(l)}}\left(f_{\theta^{(l)}}\left(x_i^{(l)}; x_1^{(l)}, \ldots, x_n^{(l)}\right)\right)
$$

### 数据和模型准备

请继续使用 TP8 的代码，并在其基础上扩展以包含基于**自注意力**的模型：与 TP8 相同，您将使用包含 50,000 条评论的 IMDB 数据集（来自 Internet Movie Database, IMDb）以及 Glove 词嵌入（请参阅实验提供的代码）。目标是对评论进行分类，判断其为“积极”（positive, pos）或“消极”（negative, neg）；性能指标为正确分类的比例（分类准确率）。

### 1. 基础模型

**问题 1**  
实现一个基于**自注意力机制**的基础模型，其中层数 $L = 3$。  
为了进行分类，可以使用均值作为最终表示。将该模型的性能与 TP9 的性能（特别是 `mean` 模块）进行比较。

### 2. 残差模型

当网络层数显著增加时，模型的学习性能会大幅下降。为解决这一问题，可以使用**残差结构**：

$$
x_i^{(l)} = g_\theta\left(x_i^{(l-1)} + f_\theta(\tilde{x}_i^{(l-1)}; \tilde{x}_1^{(l-1)}, \ldots, \tilde{x}_n^{(l-1)})\right)
$$

通常需要对输入进行归一化，以稳定网络。为此，可以采用**层归一化（Layer Normalization）**，以获得归一化后的输入 $\tilde{x}_{i}^{(l-1)}$。

**问题 2**  
实现残差模型，并将其结果与之前的模型进行比较。

### 3. 添加位置嵌入

上述模型存在一个重要问题：它没有考虑到序列的概念，这对某些任务（如情感分析）会造成困扰。例如，在处理否定句时，“I did not like” 和 “I did not know that I would enjoy” 需要不同的处理。

为了解决这一问题，自注意力模型引入了**位置嵌入（positional embeddings）**。具体来说，为每个位置 $i$ 关联一个向量 $pe_{i}$，其第 $h$ 个分量按照以下公式定义（其中 $d$ 是潜在空间的维度）：

$$
pe_{ih} =
\begin{cases}
\sin\left(\frac{i}{10000^{h/d}}\right), & \text{si } h \text{ est pair} \\
\cos\left(\frac{i}{10000^{(h-1)/d}}\right), & \text{sinon}
\end{cases}
$$

**问题 3**

在提供的代码中，有一个 `PositionalEncoding` 类，它可以为表示序列（形状为 `batch × length × k`）添加位置嵌入。在使用时需要预先指定每个批次的最大长度，以便提前计算位置嵌入向量。

**任务要求：**  
1. 为了理解该函数的作用，计算位置嵌入向量 $pe_i$ 和 $pe_j$ 的点积（对于所有 $i$ 和 $j$），并将结果以热图的形式展示。  
2. 修改第 1 题中的模型，在初始表示中添加位置嵌入（即用 $x_{i}^{(0)} + pe_i$ 替换 $x_{i}^{(0)}$）。  

还可以选择通过学习生成位置表示，而非使用解析计算的方式。更先进的模型（如 BERT）采用了**相对位置表示**而非绝对位置表示。

### 4. 添加特殊标记 CLS

Transformer 模型采用另一种方法来生成固定大小的表示以预测类别。具体来说，它在待分类序列的开头引入一个特殊标记 **CLS**，即 $x_{1}^{(0)} = x_{CLS}$。该标记的嵌入是可学习的，经过上下文建模后的表示 $x_{1}^{(L)}$ 被用于预测类别。

**问题 4**  
添加伪标记 **CLS**，并将其性能与之前的模型进行比较。

### 参考资料  
- 原始论文：[Vaswani et al., *Attention is All You Need*, 2017]  
- 博客文章：[Transformers from scratch (2019)] – 一个关于 Transformer 的优秀入门博客。